## Fine-tuning using dora

Model: deepseek
Dataset: financialPhraseBank - from finbert 
Techniques: dora
- data augmentation (to consider if data is too little)
Using ollama

Load data

### TRAINING STARTS HERE

In [2]:
from huggingface_hub import notebook_login

In [4]:
import wandb
wandb.login()

ModuleNotFoundError: No module named 'wandb'

In [1]:
%env HF_HUB_ENABLE_HF_TRANSFER=True

env: HF_HUB_ENABLE_HF_TRANSFER=True


In [ ]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"
cache_dir = "" # To add 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCasualLM, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, 
)

model = AutoModelForCausalLM.from_pretrained( # Load with vllm - todo
    model_id,
    #quantization_config=bnb_config,
    device_map="auto",
    #trust_remote_code=False
    torch_dtype=torch.bfloat16, # if A100 GPU Available     
    cache_dir=cache_dir
)

# Look at tokenizer - to adapt to dataset
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True
)

### Prepare for Dora fine-tuning

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable() # comment out if saving vram

In [ ]:
print(model)

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", # To look for for self attention & mlp without act_fn
                    "lora_magintude_vector"], # DoRa 
    lora_dropout=0.01, # To tune
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True,
)


In [ ]:
model = get_peft_model(model, peft_config)

## Tokenizer

In [ ]:
print(tokenizer)
print(tokenizer.special_tokens_map)
print(tokenizer.vocab_size)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print(tokenizer.bos_token)


In [ ]:
print(tokenizer.chat_template) # to update


messages = [
    {'role':'user', 'content':'Hello, how are you?'},
    {'role':'assistant', 'content':'I am fine, thank you!'},
    {'role':'user', 'content':'What is your name?'},
]


inputs = tokenizer.apply_chat_template(messages, tokenize=False)
print(inputs)


# TODO:
# - Check padding tokinzer, unk token to use or manually do i 